In [1]:
#One worker per core 
import findspark
findspark.init()
from pyspark import SparkContext
from pyspark.sql.session import SparkSession
from pyspark.sql.types import *

sc=SparkContext(master="local[2]")
spark = SparkSession(sc)

#from pyspark.sql.session import SparkSession
#sc = SparkContext.getOrCreate()
#spark = SparkSession(sc)
#sc.stop()
import pandas as pd 
import numpy as np 

In [2]:
flightSchema = StructType([
  StructField("DayofMonth", IntegerType(), False),
  StructField("DayOfWeek", IntegerType(), False),
  StructField("Carrier", StringType(), False),
  StructField("OriginAirportID", IntegerType(), False),
  StructField("DestAirportID", IntegerType(), False),
  StructField("DepDelay", IntegerType(), False),
  StructField("ArrDelay", IntegerType(), False),
])

In [3]:
flights=spark.read.csv("data/flights.csv", schema=flightSchema,header=True)
flights.show(5)

+----------+---------+-------+---------------+-------------+--------+--------+
|DayofMonth|DayOfWeek|Carrier|OriginAirportID|DestAirportID|DepDelay|ArrDelay|
+----------+---------+-------+---------------+-------------+--------+--------+
|        19|        5|     DL|          11433|        13303|      -3|       1|
|        19|        5|     DL|          14869|        12478|       0|      -8|
|        19|        5|     DL|          14057|        14869|      -4|     -15|
|        19|        5|     DL|          15016|        11433|      28|      24|
|        19|        5|     DL|          11193|        12892|      -6|     -11|
+----------+---------+-------+---------------+-------------+--------+--------+
only showing top 5 rows



In [4]:
airports=spark.read.csv("data/airports.csv", header=True, inferSchema=True)
airports.show(5)

+----------+-----------+-----+--------------------+
|airport_id|       city|state|                name|
+----------+-----------+-----+--------------------+
|     10165|Adak Island|   AK|                Adak|
|     10299|  Anchorage|   AK|Ted Stevens Ancho...|
|     10304|      Aniak|   AK|       Aniak Airport|
|     10754|     Barrow|   AK|Wiley Post/Will R...|
|     10551|     Bethel|   AK|      Bethel Airport|
+----------+-----------+-----+--------------------+
only showing top 5 rows



In [5]:
#Filtering columns
cities=airports.select("city","name")
cities.show(2)

+-----------+--------------------+
|       city|                name|
+-----------+--------------------+
|Adak Island|                Adak|
|  Anchorage|Ted Stevens Ancho...|
+-----------+--------------------+
only showing top 2 rows



In [6]:
#Join 2 dataframes 
flightsByOrigin=flights.join(airports,flights.OriginAirportID==airports.airport_id).groupBy("city").count()
flightsByOrigin.show(2)

+-------+-----+
|   city|count|
+-------+-----+
|Phoenix|89720|
|  Omaha|13487|
+-------+-----+
only showing top 2 rows



In [7]:
#count number of rows 
flights.count()

2702218

In [8]:
#Summary statistics 
flights.describe().show()

+-------+------------------+-----------------+-------+------------------+------------------+------------------+------------------+
|summary|        DayofMonth|        DayOfWeek|Carrier|   OriginAirportID|     DestAirportID|          DepDelay|          ArrDelay|
+-------+------------------+-----------------+-------+------------------+------------------+------------------+------------------+
|  count|           2702218|          2702218|2702218|           2702218|           2702218|           2702218|           2702218|
|   mean|15.797897875004903|3.899480352806472|   null|12742.597593162358|12743.000197985506|10.510732294729737|6.6550108096386005|
| stddev|   8.7988350691642|1.985924603367557|   null|1501.8408475102513|1501.8014309297723| 36.02975608466093|38.547584236791245|
|    min|                 1|                1|     9E|             10140|             10140|               -63|               -94|
|    max|                31|                7|     YV|             15376|          

In [9]:
#Check number of duplicates 
flights.count()-flights.dropDuplicates().count()

5561

In [10]:
#Number of rows containing missing values 
flights.count() - flights.dropDuplicates().dropna(how="any", subset=["ArrDelay", "DepDelay"]).count()

5561

In [11]:
#Impute missing values with 0
data=flights.dropDuplicates().fillna(value=0, subset=["ArrDelay", "DepDelay"])
data.count()

2696657

In [12]:
#Drop missing values 
data1=flights.dropDuplicates().dropna(how="any", subset=["ArrDelay", "DepDelay"])
data1.count()

2696657

In [13]:
data.describe().show()

+-------+------------------+------------------+-------+------------------+------------------+------------------+-----------------+
|summary|        DayofMonth|         DayOfWeek|Carrier|   OriginAirportID|     DestAirportID|          DepDelay|         ArrDelay|
+-------+------------------+------------------+-------+------------------+------------------+------------------+-----------------+
|  count|           2696657|           2696657|2696657|           2696657|           2696657|           2696657|          2696657|
|   mean|15.799071591233146|3.9004070595555906|   null|12742.454167882679|12742.858206290233|10.532424034647343|6.668734659246615|
| stddev| 8.801337925743187| 1.986466816546832|   null|1502.0468528616946|1502.0033937098096| 36.06371548529368|38.58612395657385|
|    min|                 1|                 1|     9E|             10140|             10140|               -63|              -94|
|    max|                31|                 7|     YV|             15376|         

In [14]:
data.describe.select('summary','ArrDel','DepDelay')

AttributeError: 'function' object has no attribute 'select'

In [25]:
data.corr("DepDelay", "ArrDelay")

0.9392635622308312

# Sparksql 

In [26]:
data.createOrReplaceTempView("flightData")
spark.sql("SELECT DayOfWeek, AVG(ArrDelay) AS AvgDelay FROM flightData GROUP BY DayOfWeek ORDER BY DayOfWeek").show()

+---------+------------------+
|DayOfWeek|          AvgDelay|
+---------+------------------+
|        1| 7.078911043321495|
|        2| 4.392994303553282|
|        3|  7.23574144486692|
|        4|10.776904396412583|
|        5| 8.712095985690068|
|        6| 2.143917967669759|
|        7| 5.254519736789031|
+---------+------------------+



In [28]:
data.rdd.getNumPartitions()

200

# Create View 

In [34]:
#tempory table 
#create table name airportdata from airports 
airports.createOrReplaceTempView("airportData")